Importar a tabela de vendas usando o pandas

In [ ]:
import pandas as pd

tabela_vendas = pd.read_excel('/content/drive/MyDrive/Python/Projeto1/Vendas.xlsx')
display(tabela_vendas)

Agrupar as lojas pelos valores finais somados de cada loja

In [ ]:
tabela_faturamento = tabela_vendas[['ID Loja', 'Valor Final']].groupby('ID Loja').sum()
tabela_faturamento = tabela_faturamento.sort_values(by='Valor Final', ascending=False)
display(tabela_faturamento)

Agrupar as lojas pela quantidade de produtos vendidos

In [ ]:
tabela_quantidade = tabela_vendas[['ID Loja', 'Quantidade']].groupby('ID Loja').sum()
tabela_quantidade = tabela_quantidade.sort_values(by='Quantidade', ascending=False)  # Ordena os items de quantidade de forma decresente
display(tabela_quantidade)

Calcular ticket médio. Ticket Médio = faturamento / quantidade 

In [ ]:
ticket_medio = (tabela_faturamento['Valor Final'] / tabela_quantidade['Quantidade']).to_frame()
ticket_medio = ticket_medio.rename(columns={0: 'Ticket Médio'})  # Troca o 0 do titulo da coluna por Ticket Médio
display(ticket_medio)

Função enviar e-mail:

In [62]:
def enviar_email(nome_da_loja, tabela):
  import smtplib
  import email.message

  server = smtplib.SMTP('smtp.gmail.com:587')  
  corpo_email = f"""
  <p>Prezados,</p>
  <p>Segue relatório de vendas:</p>
  {tabela.to_html()}
  <p>Qualquer dúvida, entrar em contato!</p>
  """
    
  msg = email.message.Message()
  msg['Subject'] = f"Relátorio de Vendas - {nome_da_loja}"  # Assunto do e-mail
    
  # Fazer antes (apenas na 1ª vez): Ativar Aplicativos não Seguros.
  # Gerenciar Conta Google -> Segurança -> Aplicativos não Seguros -> Habilitar
  # Caso mesmo assim dê o erro: smtplib.SMTPAuthenticationError: (534,
  # Você faz o login no seu e-mail e depois entra em: https://accounts.google.com/DisplayUnlockCaptcha
  msg['From'] = ''  # Seu gmail (Necessário ser gmail).
  msg['To'] = ''  # e-mail do destinatário, qualquer provedor
  password = ''  # Senha do seu e-mail
  msg.add_header('Content-Type', 'text/html')
  msg.set_payload(corpo_email )
    
  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()
  # Login Credentials for sending the mail
  s.login(msg['From'], password)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
  print('Email enviado')


Enviar e-mail

In [ ]:
tabela_completa = tabela_faturamento.join(tabela_quantidade).join(ticket_medio)
enviar_email("Diretoria", tabela_completa)

Enviar e-mail com faturamento individual de cada loja

In [ ]:
lojas = tabela_vendas['ID Loja'].unique()

for loja in lojas:
  linhas = tabela_vendas['ID Loja'] == loja
  tabela_loja = tabela_vendas.loc[linhas, ['ID Loja', 'Quantidade', 'Valor Final']]  # Localiza linhas e colunas
  tabela_loja = tabela_loja.groupby('ID Loja').sum()
  tabela_loja['Ticket Médio'] = tabela_loja['Valor Final'] / tabela_loja['Quantidade']  # Cria a coluna ticket médio
  enviar_email(loja, tabela_loja)
